### Selected Ngram Repeat With Youtube Link

In [24]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [25]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [26]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

In [27]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [28]:
#data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Deployment3/Data/Manuel Select"

In [29]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/4-Selected Ngram Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [30]:
# file extention
file_ext = "Fourgram"

In [31]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Result/Ngram Youtube Link Manuel"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_12.xlsx"   ########*****
#sheet = "Sheet1"  # Sheet1

In [32]:
#df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}")
df_link_selected_manuel

,video_id,start_time,end_time,text
0,3cAaBZ_Aml8,74,76,neden bu kadar çok
1,yOjfgHwn9y8,26,28,neden bu kadar çok
2,yK7PsjTRfwY,246,248,neden bu kadar çok
3,qaZnDHacYSk,515,517,neden bu kadar çok
4,WM2W7dCq3tA,981,983,neden bu kadar çok
...,...,...,...,...
328,hM5s4liNn7g,831,833,başka bir şey var
329,UtcBPDTBiko,432,434,başka bir şey var
330,zRpBEE0OT1I,165,167,başka bir şey var
331,wsUGDArhZFg,535,537,başka bir şey var


In [33]:
# disabled because of youtube tools
#df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
#df_link_selected_manuel

In [34]:
df_count = df_link_selected_manuel.groupby(["text"])[["text"]].count()
df_count.rename(columns={"text":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,text,count
0,bir şey söyleyeyim mi,9
1,gereken bir şey var,9
2,başka bir şey yok,9
3,bu ne demek oluyor,9
4,hiç bir şey yok,9
...,...,...
78,ne düşünüyorum biliyor musun,1
79,sadece sen ve ben,1
80,iyi bir fikrim var,1
81,ne demek oluyor bu,1


In [35]:
df_count = df_count[df_count["count"] > 2]

In [36]:
len(df_count)

63

In [37]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "text")
df_count_merge.sort_values(by=["count","text"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,video_id,start_time,end_time,text,count
0,4O_QCCZSoYQ,38,40,hiç böyle bir şey,9
1,o2guE9xdVt8,151,153,hiç böyle bir şey,9
2,o0vdJUDd7hQ,397,399,hiç böyle bir şey,9
3,RuVVxns8wh0,190,192,hiç böyle bir şey,9
4,wBjDafmCd_c,3408,3410,hiç böyle bir şey,9
...,...,...,...,...,...
298,YYCPETGxTuE,161,163,bir sorun mu var,3
299,AevAo6AUCiQ,113,115,bir sorun mu var,3
300,YZ0Z8t15620,18,20,benim için çok önemli,3
301,HhrBZuxjFxM,948,950,benim için çok önemli,3


In [38]:
df_count.to_excel(f"{path_folder}/{file_ext} Video List.xlsx", index=False)

In [39]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count.xlsx", index=False)   ########*****

##### Link Repeat

In [15]:
# ******  
path_folder_file = f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count_Select.xlsx"   # from youtube tool

In [16]:
#df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,video_id,start_time,end_time,text
0,nPVd4XaeM34,1032,1034,çok büyük bir
1,_J2Lhghabe0,99,101,çok büyük bir
2,RIWR28A9mMc,71,73,çok büyük bir
3,9LbSz7aJn7Y,355,357,çok büyük bir
4,0K6BSU1IqS0,562,564,çok büyük bir
...,...,...,...,...
698,0pYtgRxceX8,331,333,benim için çok
699,haycX_ZM4a4,740,742,benim için çok
700,yiSWKv_9JkE,12,14,benim için bir
701,GFVhV5BlTOA,91,93,benim için bir


In [17]:
df_count2 = df_word_group_time_loc.groupby(["text"])[["text"]].count()
df_count2.rename(columns={"text":"count"}, inplace=True)
df_count2.sort_values(by="count", ascending=False, inplace=True)
df_count2.reset_index(inplace=True)
df_count2

,text,count
0,bunu daha önce,7
1,bir şey var,7
2,her şey yolunda,7
3,her şey için,7
4,hepsi bu kadar,7
...,...,...
138,tam olarak ne,3
139,tamam o zaman,3
140,işin var burada,3
141,burada ne yapıyorsun,3


In [18]:
df_word_group_time_loc = pd.merge(df_word_group_time_loc, df_count2, how="left", on="text")
df_word_group_time_loc.drop_duplicates(inplace=True)
df_word_group_time_loc

,video_id,start_time,end_time,text,count
0,nPVd4XaeM34,1032,1034,çok büyük bir,7
1,_J2Lhghabe0,99,101,çok büyük bir,7
2,RIWR28A9mMc,71,73,çok büyük bir,7
3,9LbSz7aJn7Y,355,357,çok büyük bir,7
4,0K6BSU1IqS0,562,564,çok büyük bir,7
...,...,...,...,...,...
698,0pYtgRxceX8,331,333,benim için çok,3
699,haycX_ZM4a4,740,742,benim için çok,3
700,yiSWKv_9JkE,12,14,benim için bir,3
701,GFVhV5BlTOA,91,93,benim için bir,3


In [19]:
search_list = df_word_group_time_loc.loc[:,"text"].unique()
len(search_list)

143

In [20]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
#df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["text","start_time","end_time","sentence","video_id","video_url"])
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}"]], columns=["text","start_time","end_time","sentence","video_id"])
df_link_default

,text,start_time,end_time,sentence,video_id
0,repeat,5,7,repeat_again,Q-8I-uMUMYA


In [21]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [22]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["text"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_\
time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)

In [23]:
df_all_join_result

,video_id,start_time,end_time,text,count,web_format
0,"nPVd4XaeM34,_J2Lhghabe0,RIWR28A9mMc,9LbSz7aJn7...","1032,99,71,355,562,54,286,286,5,286,5,286,5","1034,101,73,357,564,56,288,288,8,288,8,288,8",çok büyük bir,7.0,"[play_lms_videos video_id=nPVd4XaeM34,_J2Lhgha..."
1,"SGt85ZPH1Gk,j1nouUjIHM0,MQ_nenSkjbo,4dpcKDwx1k...","777,416,1039,151,1823,2215,90,90,5,90,5,90,5","779,418,1041,153,1825,2217,92,92,8,92,8,92,8",ne olursa olsun,7.0,"[play_lms_videos video_id=SGt85ZPH1Gk,j1nouUjI..."
2,"pdGlicIifJY,BhGXfFsxgsQ,InhONwjflFE,U7oIaahi8q...","340,381,4,669,286,952,1028,669,5,669,5,669,5","342,383,6,671,288,954,1030,671,8,671,8,671,8",her şey yolunda,7.0,"[play_lms_videos video_id=pdGlicIifJY,BhGXfFsx..."
3,"BWSxerD3LDc,zX2v9speSmY,aVW36jWqI5w,dD3A1IseY0...","914,102,549,2607,43,50,687,102,5,102,5,102,5","916,104,551,2609,45,52,689,104,8,104,8,104,8",evet ben de,7.0,"[play_lms_videos video_id=BWSxerD3LDc,zX2v9spe..."
4,"5uIl38UFVh0,lD3lPxiIWXI,rcqXaFFEu1g,LsbRXTusV6...","414,345,112,2356,1063,168,68,345,5,345,5,345,5","416,347,115,2358,1065,170,70,347,8,347,8,347,8",bunu daha önce,7.0,"[play_lms_videos video_id=5uIl38UFVh0,lD3lPxiI..."
...,...,...,...,...,...,...
138,"nPVd4XaeM34,HhrBZuxjFxM,IbNzJFmtkZ0,IbNzJFmtkZ...","6747,1769,193,193,5,193,5,193,5","6749,1771,195,195,8,195,8,195,8",bir şey söylemek,3.0,"[play_lms_videos video_id=nPVd4XaeM34,HhrBZuxj..."
139,"nErPmlqYuuc,M4G_VgAYMFY,b6hXu3O2C9I,M4G_VgAYMF...","181,522,2533,522,5,522,5,522,5","183,524,2535,524,8,524,8,524,8",bir şey daha,3.0,"[play_lms_videos video_id=nErPmlqYuuc,M4G_VgAY..."
140,"kJHnaKS0Igg,2ThnEc9mxAU,YYCPETGxTuE,2ThnEc9mxA...","454,535,161,535,5,535,5,535,5","456,537,163,537,8,537,8,537,8",bir sorun mu,3.0,"[play_lms_videos video_id=kJHnaKS0Igg,2ThnEc9m..."
141,"2_3py5tiiiQ,0pYtgRxceX8,haycX_ZM4a4,haycX_ZM4a...","414,331,740,740,5,740,5,740,5","416,333,742,742,8,742,8,742,8",benim için çok,3.0,"[play_lms_videos video_id=2_3py5tiiiQ,0pYtgRxc..."


#### Copy Move And Delete

In [49]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_*.xlsx")
output_file

['Turkish_1000_Word_Youtube_Talk_Time_Join_Twogram.xlsx']

In [132]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [133]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass